In [1]:
import os

# Check these before running

# Finn's Laptop
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'
datadirbasefolder = 'derivatives/dMRI_OrigProtocol'

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, datadirbasefolder)
subjecttrackerfile = 'Subject_Tracker_for_dmri_pipeline.tsv'

###################################################################################
# USER INPUT

# Participant details
subject = '035'
session = 'MR1'

In [2]:
import os

# Define I/O folders and files
datadir = os.path.join(studydir, datadirbasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'), 'dwi')
if not os.path.exists(datadir): # then make this directory
    os.makedirs(datadir)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)
dwifilebase=f"sub-{subject}_ses-{session}_desc-preproc-inorm_dwi"
maskfilebase=f"sub-{subject}_ses-{session}_space-dwi_mask"

In [3]:

# Running amico with NODDI according to instructions
# See
# https://github.com/daducci/AMICO/wiki/Fitting-the-NODDI-model
# and more condensed in 
# https://github.com/daducci/AMICO/wiki/Decreasing-the-angular-resolution-of-the-look-up-tables#exxample-script

import amico
#amico.setup() # to be run only once

currdir = os.getcwd()
os.chdir(studydir)

# Save gradient scheme
amico.util.fsl2scheme(os.path.join(datadir, f"{dwifilebase}.bval"),os.path.join(datadir, f"{dwifilebase}.bvec"))

# Start AMICO 
ae = amico.Evaluation(datadirbasefolder, os.path.join(f'sub-{subject}', f'ses-{session}', 'dwi') )
ae.load_data( dwi_filename=os.path.join(datadir, f"{dwifilebase}.nii"), scheme_filename=os.path.join(datadir, f"{dwifilebase}.scheme"), mask_filename=os.path.join(datadir, f"{maskfilebase}.nii"), b0_thr=0 )

# Set model to NODDI
ae.set_model( "NODDI" )

# now change the radial diffusivity
dPar=1.7e-3 #1.45e-3
# use function set() to change the dPar in NODDI-model

ae.generate_kernels( regenerate=True ) # Save kernel in datadirbasefolder - tag this with name corresponding to model
ae.load_kernels()

ae.fit()
ae.save_results(path_suffix=f'dPar={dPar:.2e}')

os.chdir(currdir)


-> Writing scheme file to [ /mnt/s/Research/Projects/KPUM_NODDI/Data/derivatives/dMRI_OrigProtocol/sub-035/ses-MR1/dwi/sub-035_ses-MR1_desc-preproc-inorm_dwi.scheme ]

-> Loading data:
	* DWI signal
		- dim    = 80 x 80 x 50 x 73
		- pixdim = 2.500 x 2.500 x 2.500
	* Acquisition scheme
		- 73 samples, 2 shells
		- 1 @ b=0 , 24 @ b=1000.0 , 48 @ b=2000.0 
	* Binary mask
		- dim    = 80 x 80 x 50
		- pixdim = 2.500 x 2.500 x 2.500
		- voxels = 30742
   [ 14.1 seconds ]

-> Preprocessing:
	* Normalizing to b0... [ min=-200013.92,  mean=1.75, max=572280.00 ]
	* Keeping all b0 volume(s)
   [ 0.1 seconds ]

-> Creating LUT for "NODDI" model:


   |██████████████████████████████████████████████████████████| 100.0%


   [ 127.1 seconds ]

-> Resampling LUT for subject "sub-035/ses-MR1/dwi":


   |██████████████████████████████████████████████████████████| 100.0%


   [ 38.7 seconds ]

-> Fitting "NODDI" model to 30742 voxels (using 20 jobs):


   |██████████████████████████████████████████████████████████| 100.0%


   [ 00h 00m 15s ]

-> Saving output to "AMICO/NODDI_dPar=1.70e-03/*":
	- configuration  [OK]
	- FIT_dir.nii.gz  [OK]
	- FIT_ICVF.nii.gz  [OK]
	- FIT_OD.nii.gz  [OK]
	- FIT_ISOVF.nii.gz  [OK]
   [ DONE ]
